In [1]:
import pandas as pd 
import numpy as np
import csv
import advance_functions as advance
import RiskSLIM as slim
import stumps
from sklearn.linear_model import LogisticRegression
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_F6_fl_inter_model

no stored variable summary_F6_fl_inter_model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [3]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_F_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_F_six_month'].values

In [4]:
#### CART
depth = [1]
impurity = [0.001]
split = [2,3,4]
cart_summary = advance.CART(KY_X, KY_Y, FL_X, FL_Y, depth, split, impurity, seed=816)

### EBM
estimators = [40,60]
depth = [1]
learning_rate = [0.01, 0.03]
holdout_split = [0.9]
ebm_summary = advance.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [9]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.5230192013341647, 0.6636018045382615)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_F_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_F_six_month'].values

## columns 
cols = KY_X.columns

In [15]:
c_grid = {'C': [0.05, 0.07, 0.09]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [16]:
stumps_summary

{'best_auc': 0.5984721928880034,
 'best_params': {'C': 0.07},
 'auc_diffs': 0.0737195972900222,
 'KY_score': 0.6287380983304518}

### RiskSLIM

In [17]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.07, random_state=816).fit(FL_X, FL_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

22

In [18]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [19]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general2', 
                                max_coef = 20, 
                                max_coef_number = 10, 
                                max_runtime=200, 
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
09/17/19 @ 01:29 PM | 981 rows in lookup table
09/17/19 @ 01:29 PM | ------------------------------------------------------------
09/17/19 @ 01:29 PM | runnning initialization procedure
09/17/19 @ 01:29 PM | ------------------------------------------------------------
09/17/19 @ 01:29 PM | CPA produced 2 cuts
09/17/19 @ 01:29 PM | running naive rounding on 112 solutions
09/17/19 @ 01:29 PM | best objective value: 0.2859
09/17/19 @ 01:29 PM | rounding produced 2 integer solutions
09/17/19 @ 01:29 PM | best objective value is 0.3158
09/17/19 @ 01:29 PM | running sequential rounding on 112 solutions
09/17/19 @ 01:29 PM | best objective value: 0.2859
09/17/19 @ 01:29 PM | sequential rounding produced 2 integer solutions
09/17/19 @ 01:29 PM | best objective value: 0.2958
09/17/19 @ 01:29 PM | polishing 4 solutions
09/17/19 @ 01:29 PM | best objective value: 0.2958
09/17/19 @ 01:29 PM | polishing produced 4 integer solutions
09/17/19

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
09/17/19 @ 01:29 PM | adding 179 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2958.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [112]:
riskslim_summary['KY_auc']

[0.6192305293464628]

In [20]:
#### save results
summary_F6_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}
%store summary_F6_fl_inter_model

Stored 'summary_F6_fl_inter_model' (dict)


In [21]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Six Month\\"
results = [["Felony",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_auc'][0], 3)]]
with open(path + 'Six Month FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)